# NER in RAG

In [1]:
%pip install pandas --quiet

Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd

In [2]:
from os import environ
from dotenv import load_dotenv
load_dotenv(dotenv_path="../.env")

confluence_host = environ.get('CONFLUENCE_HOST')
confluence_token = environ.get('CONFLUENCE_TOKEN')
hf_token = environ.get('HF_TOKEN')
hf_write_token = environ.get('HF_WRITE_TOKEN')
gigachat_token = environ.get('GIGACHAT_TOKEN')
gigachat_pro_token = environ.get('GIGACHAT_PRO_TOKEN')
yandex_token = environ.get('YC_API_KEY')
openchat_host = environ.get('OPENCHAT_HOST')

## Датасет

### Загрузка датасета с Hugging Face

In [8]:
%pip install datasets --quiet

Note: you may need to restart the kernel to use updated packages.


In [3]:
from datasets import load_dataset, DatasetDict

dataset_qda = load_dataset("nizamovtimur/wikiutmn-study-gigachat")
if isinstance(dataset_qda, DatasetDict):
    train_dataset_qda = dataset_qda['train'].to_pandas()
    test_dataset_qda = dataset_qda['test'].to_pandas()
else:
    train_dataset_qda = pd.DataFrame()
    test_dataset_qda = pd.DataFrame()
len(train_dataset_qda), len(test_dataset_qda)

d:\git\virtassist\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


(355, 67)

In [4]:
train_dataset_qda

,question,document,human_answer
0,Где продлять студак?,Для восстановления студенческого билета Вам не...,
1,Когда можно получить справку о стипендии?,"По вопросу получения справки о доходах, размер...",
2,Как перевестись на другое направление на заочке?,Заявления о переводе принимаются два раза в го...,
3,Как перевестись на другое направление?,Заявления о переводе принимаются два раза в го...,
4,Могу ли я в последний момент отказаться от сме...,"Прежде, чем выбрать элективы, рекомендуем почи...",
...,...,...,...
350,Как оформляются результаты промежуточной аттес...,"проведения промежуточной аттестации, за исключ...",
351,Кто несет ответственность за правильность офор...,"проведения промежуточной аттестации, за исключ...",
352,Что происходит с оценками после прохождения пр...,". 6. 10. Из ведомости, за исключением электрон...",
353,Кто несет ответственность за правильное внесен...,". 6. 10. Из ведомости, за исключением электрон...",


In [5]:
test_dataset_qda

,question,document,human_answer
0,Я потерял магнитную карту. К кому обратиться?,Для восстановления студенческого билета Вам не...,"Если Вы потеряли магнитную карту ( пропуск, пр..."
1,Даты подачи заявления для восстановления на очку,Заявления на восстановление в Университет по о...,Заявления на восстановление в Университет по о...
2,Что делать при потере проходки?,Для восстановления студенческого билета Вам не...,"Если Вы потеряли магнитную карту ( пропуск, пр..."
3,Я хожу в фитнес-клуб. Как заменить физкультуру?,Выбор спортивных секций по Физической культуре...,Для оформления посещения спортивного зала вмес...
4,Когда мне выдадут студенческий билет после пер...,"1. Подать заявление о переводе можно лично, об...",В течение пяти рабочих дней после поступления ...
...,...,...,...
62,как закрыть физкультуру?,Выбор спортивных секций по Физической культуре...,1 ) Посещать академические занятия ( 3 балла з...
63,когда можно поменять элективы?,"Прежде, чем выбрать элективы, рекомендуем почи...","Информация, о том, как поменять электив, отпра..."
64,как получить справку о месте учёбы?,Студенты очной формы обучения оформляют справк...,Студенты очной формы обучения оформляют справк...
65,Как можно получать баллы за физру?,Выбор спортивных секций по Физической культуре...,1 ) Посещать академические занятия ( 3 балла з...


## Векторный индекс документов

### Структура БД

In [ ]:
!docker build ../db -t db

In [9]:
!docker run --name db --rm -p 5432:5432 --mount source=db_volume,target=/var/lib/postgresql/data --env-file ../.env -d db

3470db0a3cf5ae2d7e9bc8d9cc64fb84e4937b69a8e397da7ea970aba32df1c0


In [6]:
from typing import Optional
from pgvector.sqlalchemy import Vector
from sqlalchemy import Text, create_engine, select, text
from sqlalchemy.orm import DeclarativeBase, Mapped, Session, mapped_column

engine = create_engine(f"postgresql://{environ.get('POSTGRES_USER')}:{environ.get('POSTGRES_PASSWORD')}@{environ.get('POSTGRES_HOST')}/{environ.get('POSTGRES_DB')}", echo=False)

In [7]:
class Base(DeclarativeBase):
    pass

class Chunk(Base):
    __tablename__ = "chunk"
    id: Mapped[int] = mapped_column(primary_key=True)
    text: Mapped[str] = mapped_column(Text())
    embedding: Mapped[Optional[Vector]] = mapped_column(Vector(312))

In [13]:
with Session(engine) as session:
    session.execute(text('CREATE EXTENSION IF NOT EXISTS vector'))
    # session.execute(text('DROP TABLE chunk'))
    session.commit()
Base.metadata.create_all(engine)

### Заполнение документами

In [8]:
documents = pd.concat([train_dataset_qda["document"], test_dataset_qda["document"]]).drop_duplicates().reset_index(drop=True)
len(documents)

42

In [15]:
with Session(engine) as session:
    for document in documents:
        doc = Chunk(
            text=document,
        )
        session.add(doc)
    session.commit()

### Вычисление векторных представлений

#### RuSBERT-Tiny

In [16]:
from sentence_transformers import SentenceTransformer

# rusbert_model = SentenceTransformer('cointegrated/rubert-tiny2', device="cpu")
rusbert_model = SentenceTransformer('nizamovtimur/rubert-tiny2-wikiutmn', device="cpu")
len(rusbert_model.encode("мама мыла раму"))

312

In [17]:
with Session(engine) as session:
   chunks = session.scalars(select(Chunk).order_by(Chunk.id)).all()
   for chunk in chunks:
      chunk.sbert_312 = rusbert_model.encode(chunk.text)
      session.add(chunk)
      session.flush()
   session.commit()

In [18]:
def answer_sbert_312(question):
    with Session(engine) as session:
        return session.scalars(select(Chunk)
                        .order_by(Chunk.sbert_312.cosine_distance(
                            rusbert_model.encode(question)
                            )).limit(1)).first().text

answer_sbert_312("Как поменять физкультуру?")

'Выбор спортивных секций по Физической культуре будет проходить в ИС Модеус во вкладке " Выбор модулей ". Вам нужно будет выбрать 2 интересующие Вас спортивные секции, которые будут проходить каждую неделю в одно и то же время. Ограничения : 1 ) Записаться можно не более чем на 2 занятия в неделю 2 ) Нельзя записываться на два занятия подряд. Вас могут не допустить на занятие, если Вы были на предыдущей паре и / или уже посетили два занятия за неделю. При этом расписание на наличие конфликтов Вы проверяете самостоятельно в соответствии с Вашим расписанием в ИС Модеус и расписанием спортивных секций ( во вложенных файлах ). Ваш выбор пролонгируется до конца семестра, однако в любой момент Вы можете его изменить, отписавшись от одной секции и записавшись на другую. ВАЖНО! Студент, пропустивший два занятия подряд, будет отписан автоматически. Выбор Физической культуры откроется 06. 09. 2023 и будет открыт до конца семестра. Для успешной аттестации по дисциплине « Физическая культура : эле

### Выбор нужного фрагмента через векторный индекс

In [19]:
test_dataset_qda["sbert_312"] = test_dataset_qda["question"].apply(answer_sbert_312)
test_dataset_qda.head(5)

,question,document,human_answer,sbert_1024
0,Я потерял магнитную карту. К кому обратиться?,Для восстановления студенческого билета Вам не...,"Если Вы потеряли магнитную карту ( пропуск, пр...",Для восстановления студенческого билета Вам не...
1,Даты подачи заявления для восстановления на очку,Заявления на восстановление в Университет по о...,Заявления на восстановление в Университет по о...,Заявления на восстановление в Университет по о...
2,Что делать при потере проходки?,Для восстановления студенческого билета Вам не...,"Если Вы потеряли магнитную карту ( пропуск, пр...",Для восстановления студенческого билета Вам не...
3,Я хожу в фитнес-клуб. Как заменить физкультуру?,Выбор спортивных секций по Физической культуре...,Для оформления посещения спортивного зала вмес...,Выбор спортивных секций по Физической культуре...
4,Когда мне выдадут студенческий билет после пер...,"1. Подать заявление о переводе можно лично, об...",В течение пяти рабочих дней после поступления ...,на договорное место ). В течение пяти рабочих ...


### Выбор лучшего алгоритма

#### Accuracy

In [20]:
for column in test_dataset_qda.columns[2:]:
    print(column, sum(test_dataset_qda[column].apply(lambda x: "" if x is None else x) == test_dataset_qda.document) / len(test_dataset_qda.document))

human_answer 0.31343283582089554
sbert_1024 0.6119402985074627


#### ROUGE-L

In [53]:
%pip install rouge --quiet

Note: you may need to restart the kernel to use updated packages.


In [21]:
from rouge import Rouge
rouge = Rouge()

for column in test_dataset_qda.columns[2:]:
    print(column, rouge.get_scores(test_dataset_qda[column].apply(lambda x: "-" if x is None else x), test_dataset_qda["document"], avg=True)['rouge-l'])

human_answer {'r': 0.6170558241367031, 'p': 0.9958196833933283, 'f': 0.7133136761037521}
sbert_1024 {'r': 0.7993599506385278, 'p': 0.8203655026172052, 'f': 0.8004825652323815}
